In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import seaborn as sns
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
df = pd.read_csv('Crop_recommendation_Dataset.csv')

In [ ]:
df

In [ ]:
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

In [ ]:
# Save the modified DataFrame to a new CSV file
df.to_csv('CR-encoded_data.csv')

In [ ]:
data = pd.read_csv('CR-encoded_data.csv')

In [ ]:
data

In [ ]:
X = data[['N','P','K','temperature','humidity','ph','rainfall']]
Y = data['label_encoded']

In [ ]:
X

In [ ]:
Y

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Create an SVR model
#svr = SVR(kernel='linear', C=1.0)  
svr = SVR(kernel='sigmoid', C=1.0, gamma='scale')


In [ ]:
# Train the SVR model on the training data
svr.fit(X_train, Y_train)

In [ ]:
# Make predictions on the test data
Y_pred = svr.predict(X_test)

In [ ]:
# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

In [ ]:
print(mse)
print(r2)

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(Y_test, Y_pred, color='blue', label='Actual')
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linestyle='--', lw=2, label='Predicted')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Values")
plt.legend()
plt.show()


In [ ]:
# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
# Input values and get predictions
print("Enter the following values for prediction:")
nitrogen = float(input("Nitrogen: "))
phosphorous = float(input("Phosphorous: "))
potassium = float(input("Potassium: "))
temperature = float(input("Temperature: "))
humidity = float(input("Humidity: "))
ph = float(input("pH levels"))
rainfall = float(input("Rainfall (in centimeters)"))

# Create a DataFrame with the input values
input_data = pd.DataFrame({
    'N': [nitrogen],
    'P': [phosphorous],
    'K': [potassium],
    'temperature': [temperature],
    'humidity': [humidity],
    'ph': [ph],
    'rainfall': [rainfall]
})

In [ ]:
# Use the trained SVR model to make a prediction
predicted_crop_encoded = svr.predict(input_data)
rounded_predictions = np.round(predicted_crop_encoded)

# Get the classes from the label encoder
encoded_classes = label_encoder.classes_

# Check if the predicted label is within the range of classes
if 0 <= rounded_predictions[0] < len(encoded_classes):
    predicted_crop_name = encoded_classes[int(rounded_predictions[0])]
    print(f"Predicted crop Name: {predicted_crop_name}")
else:
    print("Predicted label is outside the range of classes.")

# Convert rounded_predictions to integer
rounded_predictions = int(rounded_predictions[0])  # Extract the first (and only) element from the array

# Decode the predicted crop name
predicted_crop_name = label_encoder.inverse_transform([rounded_predictions])

print(f"Predicted crop Name: {predicted_crop_name[0]}")

In [ ]:
# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")